In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_json("../input/train.json")

In [3]:
train.head()

,band_1,band_2,id,inc_angle,is_iceberg
0,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",dfd5f913,43.9239,0
1,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",e25388fd,38.1562,0
2,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",58b2aaa0,45.2859,1
3,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",4cfc3a18,43.8306,0
4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",271f93f4,35.6256,0


In [4]:
len(train.band_1[0])

5625

In [5]:
train.shape

(1604, 5)

In [6]:
train = train[train['inc_angle'] != 'na']

In [7]:
train_label = train.is_iceberg
train_label = np.array(train_label, dtype='float')

In [8]:
train_label

array([0., 0., 1., ..., 1., 0., 1.])

In [9]:
train_data = train.drop(columns=['is_iceberg', 'id'])

In [10]:
train_data = train_data[train_data['inc_angle'] != 'na']

In [11]:
train_data.shape

(1471, 3)

In [12]:
rows = train_data.shape[0]

In [13]:
train_data.head()

,band_1,band_2,inc_angle
0,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",43.9239
1,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",38.1562
2,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",45.2859
3,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",43.8306
4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",35.6256


In [14]:
train_data.tail()

,band_1,band_2,inc_angle
1504,"[-20.528059, -22.144562, -26.714188, -24.64341...","[-25.471151, -27.77906, -28.993053, -28.165241...",45.2814
1505,"[-15.371544, -14.431547, -15.970677, -18.76996...","[-22.540173, -23.125895, -21.643795, -24.43109...",35.7829
1506,"[-23.945009, -23.316914, -25.883354, -22.36160...","[-26.738250999999998, -25.616005, -23.945156, ...",34.4721
1507,"[-26.656988, -26.972933, -27.300798, -26.05789...","[-23.135164, -24.474159, -23.559067, -25.23004...",37.9814
1508,"[-16.241388, -17.401228, -18.385656, -19.23677...","[-26.05958, -24.065521, -23.421896, -25.257374...",35.2987


In [15]:
angle = np.array([x[2] for x in train_data.values])

In [17]:
len(angle), len(train_data.values)

(1471, 1471)

In [18]:
data_bands = np.array([np.concatenate((np.array(x[0]), np.array(x[1]), (np.array(x[0]) * a), (np.array(x[1]) * a))) for x, a in zip(train_data.values, angle)])

AxisError: axis 1 is out of bounds for array of dimension 1

In [51]:
data_bands.shape

(1471, 22500)

In [53]:
scaler = MinMaxScaler(feature_range=(0,2))

In [54]:
data_bands = scaler.fit_transform(data_bands)

In [55]:
data_bands[:3]

array([[0.38098953, 0.47526341, 0.32780459, ..., 0.49301309, 0.62871997,
        0.45176514],
       [1.30256145, 1.19798695, 1.10282477, ..., 0.94766608, 1.0647203 ,
        0.94858258],
       [0.57399672, 0.62593279, 0.54188313, ..., 0.85666554, 0.76705688,
        0.62940587]])

In [57]:
imgs_bands = np.reshape(data_bands, (rows, 75, 75, 4))

In [77]:
print(imgs_bands.shape, train_label.shape)

(1471, 75, 75, 4) (1604,)


In [75]:
from sklearn.model_selection import train_test_split

In [82]:
X, X_test, y, y_test = train_test_split(imgs_bands, train_label, test_size=0.30, stratify=train_label)

In [83]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (8, 8), activation='relu', input_shape=(75, 75, 4)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (4, 4), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(.1))
model.add(layers.Conv2D(64, (2, 2), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(.2))
model.add(layers.Dense(1, activation='sigmoid'))

/home/kleber/anaconda3/envs/tensorflow-cpu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/kleber/anaconda3/envs/tensorflow-cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [84]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X, y, epochs=30, batch_size=8, validation_data=(X_test, y_test))

Train on 1029 samples, validate on 442 samples
Epoch 1/30
1029/1029 [==============================] - 18s 18ms/step - loss: 0.7497 - acc: 0.4985 - val_loss: 0.6924 - val_acc: 0.5113
Epoch 2/30
1029/1029 [==============================] - 18s 18ms/step - loss: 0.7043 - acc: 0.5112 - val_loss: 0.6996 - val_acc: 0.5113
Epoch 3/30
 568/1029 [===============>..............] - ETA: 7s - loss: 0.7048 - acc: 0.4982

KeyboardInterrupt: 

In [55]:
from keras.models import model_from_json
import os

In [56]:
model_json = model.to_json()
with open('./model/covnet-v2.json', 'w') as json_file:
    json_file.write(model_json)

In [57]:
model.save_weights('./model/covnet-v2.h5')

In [58]:
test = pd.read_json('../input/test.json')

In [59]:
test.head()

,band_1,band_2,id,inc_angle
0,"[-15.863251, -15.201077, -17.887735, -19.17248...","[-21.629612, -21.142353, -23.908337, -28.34524...",5941774d,34.966400
1,"[-26.058969497680664, -26.058969497680664, -26...","[-25.754207611083984, -25.754207611083984, -25...",4023181e,32.615072
2,"[-14.14109992980957, -15.064241409301758, -17....","[-14.74563980102539, -14.590410232543945, -14....",b20200e4,37.505433
3,"[-12.167478, -13.706167, -16.54837, -13.572674...","[-24.32222, -26.375538, -24.096739, -23.8769, ...",e7f018bb,34.473900
4,"[-23.37459373474121, -26.02718162536621, -28.1...","[-25.72234344482422, -27.011577606201172, -23....",4371c8c3,43.918874


In [60]:
test.shape

(8424, 4)

In [61]:
test_bands = np.array([x[0] + x[1] for x in test.values])

In [62]:
test_bands = scaler.transform(test_bands)

In [63]:
test_bands.shape

(8424, 11250)

In [64]:
test_bands = np.reshape(test_bands, (8424, 75, 75, 2))

In [65]:
ids, is_iceberg = [], []

In [66]:
for row in range(len(test_bands)):
    img = test_bands[row]
    ids.append(test.id[row])
    pred = model.predict(np.array([img]))
    is_iceberg.append(pred[0][0])

print(ids[:10], is_iceberg[:10])

['5941774d', '4023181e', 'b20200e4', 'e7f018bb', '4371c8c3', 'a8d9b1fd', '29e7727e', '92a51ffb', 'c769ac97', 'aee0547d'] [0.8325968, 0.4048494, 1.0, 0.9964868, 0.37934244, 1.0, 5.3528324e-06, 0.9994628, 2.2349452e-18, 8.520841e-19]


In [67]:
df_result = pd.DataFrame({'id': ids,'is_iceberg': is_iceberg})

In [68]:
df_result.head()

,id,is_iceberg
0,5941774d,0.832597
1,4023181e,0.404849
2,b20200e4,1.000000
3,e7f018bb,0.996487
4,4371c8c3,0.379342


In [69]:
df_result.to_csv('submission_v2.csv', encoding='utf-8', float_format='%.16f', index=False)

In [70]:
df_result.is_iceberg.mean()

0.5692961954964327

In [71]:
df_result.is_iceberg.std()

0.4562533591068056